In [1]:
import pandas as pd

# Import Data
kiwo_data = pd.read_csv('kiwo.csv', parse_dates=['Datum'])
umsatzdaten_gekuerzt_data = pd.read_csv('umsatzdaten_gekuerzt.csv', parse_dates=['Datum'])
wetter_data = pd.read_csv('wetterdaten.csv', parse_dates=['Datum'])

# Merge the dataframes
all_data = pd.merge(kiwo_data, umsatzdaten_gekuerzt_data, on="Datum")
all_data = pd.merge(all_data, wetter_data, on="Datum")

# Inspect the data: print the first 5 rows of the dataframe, inspect columns
print(all_data.head())
print(all_data.columns)

       Datum  KielerWoche  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0 2014-06-21            1            1  162.410673         6.0     15.5125   
1 2014-06-21            1            2  691.302130         6.0     15.5125   
2 2014-06-21            1            3  266.758834         6.0     15.5125   
3 2014-06-21            1            4   55.378436         6.0     15.5125   
4 2014-06-21            1            5  398.075089         6.0     15.5125   

   Windgeschwindigkeit  Wettercode  
0                   18        61.0  
1                   18        61.0  
2                   18        61.0  
3                   18        61.0  
4                   18        61.0  
Index(['Datum', 'KielerWoche', 'Warengruppe', 'Umsatz', 'Bewoelkung',
       'Temperatur', 'Windgeschwindigkeit', 'Wettercode'],
      dtype='object')
